In [1]:
import h2o
from h2o.estimators import H2ORandomForestEstimator
h2o.init()
from h2o.frame import H2OFrame
import pandas as pd
import numpy as np
import math
import scipy.stats

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM JBR-11.0.13.7-1751.21-jcef (build 11.0.13+7-b1751.21, mixed mode)
  Starting server from C:\Users\hecto\miniconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\hecto\AppData\Local\Temp\tmpuxf2f052
  JVM stdout: C:\Users\hecto\AppData\Local\Temp\tmpuxf2f052\h2o_hecto_started_from_python.out
  JVM stderr: C:\Users\hecto\AppData\Local\Temp\tmpuxf2f052\h2o_hecto_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Denver
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_hecto_43qnfl
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,15.93 Gb
H2O_cluster_total_cores:,20
H2O_cluster_allowed_cores:,20
H2O_cluster_status:,"locked, healthy"


In [2]:
heart_data = pd.read_excel(r"C:\Users\hecto\OneDrive\Documents\Jupyter Notebook\heart_data.xlsx", sheet_name='Sheet1')
heart_data['HeartDisease'] = heart_data['HeartDisease'].astype('category')
h2o_df = H2OFrame(heart_data)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [3]:
heart_data.head()
h2o_df.head()

Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
40,M,ATA,140,289,0,Normal,172,N,0,Up,0
49,F,NAP,160,180,0,Normal,156,N,1,Flat,1
37,M,ATA,130,283,0,ST,98,N,0,Up,0
48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
54,M,NAP,150,195,0,Normal,122,N,0,Up,0
39,M,NAP,120,339,0,Normal,170,N,0,Up,0
45,F,ATA,130,237,0,Normal,170,N,0,Up,0
54,M,ATA,110,208,0,Normal,142,N,0,Up,0
37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1
48,F,ATA,120,284,0,Normal,120,N,0,Up,0


In [4]:
#Creating Numpy Arrays for manipulation
age = np.array(heart_data.loc[:,"Age"])
sex = np.array(heart_data.loc[:,"Sex"])
chest_pain = np.array(heart_data.loc[:,"ChestPainType"])
blood_pressure = np.array(heart_data.loc[:,"RestingBP"])
cholesterol = np.array(heart_data.loc[:,"Cholesterol"])
blood_sugar = np.array(heart_data.loc[:,"FastingBS"])
ECG = np.array(heart_data.loc[:,"RestingECG"])
max_hr = np.array(heart_data.loc[:,"MaxHR"])
angina = np.array(heart_data.loc[:,"ExerciseAngina"])
oldpeak = np.array(heart_data.loc[:,"Oldpeak"])
ST_Slope = np.array(heart_data.loc[:,"ST_Slope"])
HeartDisease = np.array(heart_data.loc[:,"HeartDisease"])



In [5]:
#Exploratory Statistics - Examining relationships between variables


#Exploration 1: Does a linear relationship exist between age & blood pressure?
#Here, I will define a function to calculate the Pearson Correlation Coefficient for practice, verify it against the built in 
#fucntion in sklearn, and use it to explore relationships in the data

def pearson_coeff(ds1, ds2):
    
    term1 = []
    for i in range(len(ds1)):
        term1.append((ds1[i] - np.mean(ds1))*(ds2[i] - np.mean(ds2)))
    term1 = np.sum(term1)

    term2 = []
    for i in range(len(ds1)):
        term2.append((ds1[i] - np.mean(ds1))**2)
    term2 = np.sum(term2)

    term3 = []
    for i in range(len(ds2)):
        term3.append((ds2[i] - np.mean(ds2))**2)
    term3 = np.sum(term3)

    term4 = math.sqrt(term3 * term2)
    
    return (term1/term4)

if round(pearson_coeff(age, blood_pressure), 5) == round(scipy.stats.pearsonr(age, blood_pressure)[0], 5):
    print("Success!")

Success!


In [6]:
#Here, we examine the correlation between dependent variables out of interest, and to see if we can reduce input dimensionality 
print("The correlation coeff. for age and blood pressure is {}{}".format(pearson_coeff(age, blood_pressure), "\n"))
print("The correlation coeff. for age and cholesterol is {}{}".format(pearson_coeff(age, cholesterol), "\n"))
print("The correlation coeff. for cholesterol and blood pressure is {}{}".format(pearson_coeff(blood_pressure, cholesterol), "\n"))
print("The correlation coeff. for max HR and age is {}{}".format(pearson_coeff(max_hr, age), "\n"))

The correlation coeff. for age and blood pressure is 0.25547922593487304

The correlation coeff. for age and cholesterol is -0.09574379248759145

The correlation coeff. for cholesterol and blood pressure is 0.10096232102095347

The correlation coeff. for max HR and age is -0.38137416703608007



In [7]:
#Here, we examine the relationship between some dependent variables and the incidence of heart failure. I hypothesize that
#cholesterol have a direct correlation with Heart Disease, and max heart rate will have an inverse correlation

print("The correlation coeff. for max heart rate and heart disease incidence: " + str(pearson_coeff(max_hr, HeartDisease)))
print("\n")
print("The correlation coeff, for cholesterol and heart disease incidence:", pearson_coeff(cholesterol, HeartDisease))

#Interestingly, cholsterol and HeartDisease have an inverse correlation. 

The correlation coeff. for max heart rate and heart disease incidence: -0.40098263327778494


The correlation coeff, for cholesterol and heart disease incidence: -0.23316248058476585


In [8]:
#Now that we have performed some basic exploration, we will now create ML models to predict heart disease incidence. Our goal
#will be over 80% accuracy or higher

features = h2o_df.columns[:-1]  # Assuming the target variable is the last column
target = h2o_df.columns[-1]
h2o_df[target] = h2o_df[target].asfactor()
train, test = h2o_df.split_frame(ratios=[0.8])


model = H2ORandomForestEstimator(ntrees=100, max_depth=20)

#Create a dstributed random foh2o.randomForest
drf = H2ORandomForestEstimator(
                                ntrees = 20,
                                max_depth = 20,
)                            #categorical_encoding = "one_hot_explicit")
    

#Now, we train our DRF model
drf.train(x=features,
          y=target,
          training_frame=train,
          validation_frame=test)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: DRF_model_python_1684369343609_1


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    20                 20                          25904                  11           17           13.6          87            120           98.3

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.11720800983710808
RMSE: 0.34235655366460865
LogLoss: 1.2011841247429698
Mean Per-Class Error: 0.1600023192943547
AUC: 0.9070325353511194
AUCPR: 0.894614650891259
Gini: 0.8140650707022388

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4599999904632568
       0    1    Error    Rate
-----  ---  ---  -------  -------------
0      268  71   0.2094   (71.0/339.0)
1      45   362  0.1106   (45.0/407.0)
Total  313  433  0.1555   (116.0/746.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.46         0.861905  45
max f2                       0.2          0.913729  66
max f0point5                 0.611111     0.869333  35
max accuracy                 0.553571     0.848525  41
max precision                0.9          0.927461  7
max recall                   0            1         80
max specificity              1            0.958702  0
max absolute_mcc             0.553571     0.694952  41
max min_per_class_accuracy   0.555556     0.840708  40
max mean_per_class_accuracy  0.553571     0.847872  41
max tns                      1            325       0
max fns                      1            248       0
max fps                      0            339       80
max tps                      0            407       80
max tnr                      1            0.958702  0
max fnr                      1            0.609337  0
max fpr                      0            1         80
max tpr                      0            1         80

Gains/Lift Table: Avg response rate: 54.56 %, avg score: 53.58 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.231903                    1                  1.68459   1.68459            0.919075         1          0.919075                    1                   0.390663        0.390663                   68.4595   68.4595            0.349365
2        0.327078                    0.857143           1.70384   1.6902             0.929577         0.882533   0.922131                    0.965819            0.162162        0.552826                   70.3845   69.0196            0.496778
3        0.420912                    0.75               1.62345   1.67532            0.885714         0.791404   0.914013                    0.926937            0.152334        0.70516                    62.3447   67.5316            0.625514
4        0.510724                    0.6                1.31314   1.61163            0.716418         0.659893   0.879265                    0.879976            0.117936        0.823096                   31.3139   61.1626            0.687403
5        0.604558                    0.428571           0.811723  1.48747            0.442857         0.502549   0.81153                     0.821396            0.0761671       0.8992

In [9]:
#For conciseness, we will call the below function to get a report of the model performance on the testing data:

print("Training Accuracy: {}{}".format(drf.accuracy()[0][0], "\n"))
print("Validation Accuracy: {}".format(drf.accuracy()[0][1]))

#Success! We have achieved an accuracy above 80%


Training Accuracy: 0.5535714328289032

Validation Accuracy: 0.8485254691689008


In [10]:
#Now that we've trained the model, we can use it to predict 3 datapoints I removed from the original dataset as a final form of
#validation that we can directly observe, rather than having the validation be hidden by the model API

patient1 = np.array([57, "M", "ASY", 130, 131, 0, "Normal", 115, "Y", 1.2, "Flat"])
patient2 = np.array([57, "F", "ATA", 130, 236, 0, "LVH", 174, "N", 0, "Flat"])
patient3 = np.array([38, "M", "NAP", 138, 175, 0, "Normal", 173, "N", 0, "Up"])


#Disclaimer, I am new to h2o so I am building a pandas dataframe first, then converting to an h2o dataframe
p1 = {
    'Age': [57],
    'Sex': ['M'],
    'ChestPainType': ['ASY'],
    'RestingBP': [130],
    'Cholesterol': [131],
    'FastingBS': [0],
    'RestingECG': ['Normal'],
    'MaxHR': [115],
    'ExerciseAngina': ['Y'],
    'Oldpeak': [1.2],
    'ST_Slope': ['Flat']
}


p2 = {
    'Age': [57],
    'Sex': ['F'],
    'ChestPainType': ['ATA'],
    'RestingBP': [130],
    'Cholesterol': [236],
    'FastingBS': [0],
    'RestingECG': ['LVH'],
    'MaxHR': [174],
    'ExerciseAngina': ['N'],
    'Oldpeak': [0],
    'ST_Slope': ['Flat']
}


p3 = {
    'Age': [38],
    'Sex': ['M'],
    'ChestPainType': ['NAP'],
    'RestingBP': [138],
    'Cholesterol': [175],
    'FastingBS': [0],
    'RestingECG': ['Normal'],
    'MaxHR': [173],
    'ExerciseAngina': ['N'],
    'Oldpeak': [0],
    'ST_Slope': ['Up']
}

df1 = pd.DataFrame(p1)
df2 = pd.DataFrame(p2)
df3 = pd.DataFrame(p3)

patient1 = H2OFrame(df1)
patient2 = H2OFrame(df2)
patient3 = H2OFrame(df3)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
#First, lets make a prediction for patient 1. Note that this patient has heart disease, so we are expecting a prediction of "1"
drf.predict(patient1)

#Success! We have correctly diagnosed patient 1 with a high degree of confidence

drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict,p0,p1
1,0.05,0.95


In [15]:
#Now, lets make a prediction for patient 2. Note that this patient also has heart disease
drf.predict(patient2)

#Unfortunately, our model returned a false negative, which is highly undesirable in the context of healthcare diagnoses. 
#However, the model did note that the patient does have a nontrivial propbability of having heart disease. Given that this
#model is simply for demonstration, I will not further attempt to tune it, but we should be aware of the drawbacks of
#overreliance on Machine Learning for critical applications. On the flip side, we should also remain cognizant that we can learn
#valuable insights from ML models if we are diligent. For example, perhaps we could learn that a certain combination of
#symptomotologies paradoxically tends to cause false negatives when doctors are diagnosing heart disease, and we can use
#this knowledge to better equip healthcare providers with tools to treat their patients.

drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict,p0,p1
0,0.85,0.15


In [14]:
#Finally, we predict patient 3. This patient does NOT have heart disease, so we expect a prediction of "0"
drf.predict(patient3)

#Another success! We have correctly diagnosed patient 3 with a high degree of confidence

drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict,p0,p1
0,1,0


In [18]:
#Random Forests can have a number of drawbacks when working with categorical data like we are, and are also occasionally 
#criticized for their computational complexity and low interpretability. As such, lets try to create a simpler model with 
#comparable accuracy to our Random Forest. For this, we will use Logistic Regression from the sklearn API. We will also drop
#the one-hot encoding from our prediction variables, and see how this affects the accuracy. 

import sklearn.linear_model
import sklearn.model_selection

#Create a master array of input variables
master = np.concatenate((age.reshape(-1,1), blood_pressure.reshape(-1,1), cholesterol.reshape(-1,1), max_hr.reshape(-1,1)), axis = 1)
print(master.shape)

#Split the training and testing data
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(master, HeartDisease, test_size = 0.3)

#Instantiate our model
myClassifier = sklearn.linear_model.LogisticRegression()

#Train the model
myClassifier.fit(x_train, y_train)



(915, 4)


LogisticRegression()

In [19]:
#Now, lets examine our accuracy on the test data.

print("Testing Accuracy: {}".format(myClassifier.score(x_test, y_test)))

Testing Accuracy: 0.6945454545454546


In [20]:
#We've successfully built a Logistic Regression model with a reasonable degree of accuracy. 
#Lets try it out on our hypothetical patients.


patient_1 = np.array([57, 130, 131, 115])
patient_2 = np.array([57, 130, 236, 174])
patient_3 = np.array([38, 138, 175, 173])

if myClassifier.predict(patient_1.reshape(1,-1))[0] == 0:
    print("Patient 1 does not have heart disease")
else:
    print("Patient 1 has heart disease")
    
if myClassifier.predict(patient_2.reshape(1,-1))[0] == 0:
    print("Patient 2 does not have heart disease")
else:
    print("Patient 2 has heart disease")    
    
if myClassifier.predict(patient_3.reshape(1,-1))[0] == 0:
    print("Patient 3 does not have heart disease")
else:
    print("Patient 3 has heart disease")

#As visible below, we have correctly predicted the outcomes for 2 out of the 3 patients, similar to the Random Forest model. 
#Note that based on the respective testing accuracy of the models, we would expect the Random Forest to perform better
#over the course of 1000 patients despite the identical performance we see in this notebook. 

#This concludes my personal project. Thanks for reading, and all feedback is appreciated!

Patient 1 has heart disease
Patient 2 does not have heart disease
Patient 3 does not have heart disease
